In [ ]:
!pip3 install requests lxml pandas --user

In [ ]:
import requests
import lxml.html as lh
import pandas as pd

In [ ]:
# country_names = ['SRI LANKA','CHINA 1','CHINA 2','CHINA - HONG KONG','CHINA - MACAO','SPAIN','SPAIN - CANARY ISLAND','SPAIN - CEUTA AND MELILLA',
#                 'FRANCE 1','GERMANY','INDIA','INDIA - NORTH','INDIA - SOUTH','ITALY','JAPAN','SPAIN- JUAN CARLOS','USA 1','USA 2','THAILAND',
#                 'AUSTRALIA 1','AUSTRALIA 2','AUSTRALIA 3','AUSTRALIA 4','AUSTRALIA 5','AUSTRALIA 6','AUSTRALIA 7','AUSTRALIA 8',
#                 'AUSTRIA', 'FRANCE 2', 'FRENCH GUIANA','FRENCH POLYNESIA 1','FRENCH POLYNESIA 2','FRENCH POLYNESIA 3','FRENCH POLYNESIA 4']
# country_codes =  [2120, 7100,2250,2150,2160, 6140,1010,1024,7130,7030,7108,2100,2110,7134,7110,7102,7040,7042,2210,5320,5250,5260,
#                  5270,5280,5290,5300,5310,6180,6130,3050,5210,5170,5190,5200]

scrape_period = {'01' : ['01','02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30','31'],
                 '02' : ['01','02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29'],
                 '03' : ['01','02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19']
                }
country_code_dict = {'SRI LANKA': 2120, 'CHINA 1': 7100, 'CHINA 2': 2250, 'CHINA - HONG KONG': 2150,
                     'CHINA - MACAO': 2160,'SPAIN': 6140,'SPAIN - CANARY ISLAND': 1010,'SPAIN - CEUTA AND MELILLA': 1024,
                     'FRANCE 1': 7130,'GERMANY': 7030,'INDIA': 7108,'INDIA - NORTH': 2100,'INDIA - SOUTH': 2110,
                     'ITALY': 7134,'JAPAN': 7110,'SPAIN- JUAN CARLOS': 7102,'USA 1': 7040,'USA 2': 7042,'THAILAND': 2210,
                     'AUSTRALIA 1': 5320,'AUSTRALIA 2': 5250,'AUSTRALIA 3': 5260,'AUSTRALIA 4': 5270,
                     'AUSTRALIA 5': 5280,'AUSTRALIA 6': 5290,'AUSTRALIA 7': 5300,'AUSTRALIA 8': 5310,'AUSTRIA': 6180,
                     'FRANCE 2': 6130,'FRENCH GUIANA': 3050,'FRENCH POLYNESIA 1': 5210,'FRENCH POLYNESIA 2': 5170,
                     'FRENCH POLYNESIA 3': 5190,'FRENCH POLYNESIA 4': 5200
                    }

In [ ]:
def scrape_page(country_code, start_date, start_month, start_year, end_day, end_month,end_year):
    url="http://www.meteomanz.com/sy2?l=1&cou="+str(country_code) + "&ind=00000&d1=" +str(start_day) + "&m1=" + start_month + "&y1="+ str(start_year)+'&d2='+str(end_day) + '&m2=' + str(end_month) +'&y2='+ str(end_year) 
    print(url)
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')
    #Create empty list
    col=[]
    i=0#For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
#         print('%d:"%s"'%(i,name))
        col.append((name,[]))
    #Since out first row is the header, data is stored on the second row onwards
    for j in range(1,len(tr_elements)):
        #T is our j'th row
        T=tr_elements[j]

        #If row is not of size 10, the //tr data is not from our table 
        if len(T)!=10:
            break

        #i is the index of our column
        i=0

        #Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content() 
            #Check if row is empty
            if i>0:
            #Convert any numerical value to integers
                try:
                    data=int(data)
                except:
                    pass
            #Append the data to the empty list of the i'th column
            col[i][1].append(data)
            #Increment i for the next column
            i+=1
    Dict={title:column for (title,column) in col}
    df=pd.DataFrame(Dict)
    return df

In [ ]:
start_year = 2020
end_year = 2020
all_countries_data= []
for country_name in country_code_dict:
    country_code = country_code_dict[country_name]
    country_df = []
    for month in scrape_period:
        start_month = month
        end_month = month
        for day in scrape_period[month]:
            start_day = day
            end_day = day
            day_df = scrape_page(country_code, start_day, start_month, start_year, end_day, end_month,end_year)
            country_df.append(day_df)
#         print(country_df)
    country_df = pd.concat(country_df,ignore_index=True)
    country_df['country'] = country_name
    all_countries_data.append(country_df)

In [ ]:
all_countries_data = pd.concat(all_countries_data)

In [ ]:
all_countries_data.to_csv("daily_weather_data_selected_countries.csv")